In [2]:
!pip -V

pip 23.3.1 from /root/feature-map-experiments/.cenv/lib/python3.10/site-packages/pip (python 3.10)


# .init

In [3]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [18]:
import os
import sys
from pathlib import Path
workspace_dir = Path.cwd()
if not (workspace_dir / '.git').exists():
    workspace_dir = workspace_dir.parent


module_path = str(workspace_dir.resolve()) # os.path.abspath(os.path.join('..'))
print(f'workspace dir: {module_path}')

# Fixes a periodic issue with locating local packages.
if module_path not in sys.path:
    sys.path.append(module_path)

workspace dir: /root/feature-map-experiments


In [6]:
import s3fs
fs = s3fs.S3FileSystem(
    anon=False,
    use_ssl=True,
    endpoint_url='https://s3.us-west-002.backblazeb2.com',
    key='002fcb156df54ab0000000003',
    secret = 'K002eQNcQEL2F612vYfbBOFFFAwzf/g',

    # client_kwargs={
    #     "aws_access_key_id": '002fcb156df54ab0000000003',
    #     "aws_secret_access_key": 'K002eQNcQEL2F612vYfbBOFFFAwzf/g',
    #     "endpoint_url": 'https://s3.us-west-002.backblazeb2.com',
    #     "verify": False}
        
        )

print(fs.ls("s3://RA-CYT-PAP"))

['RA-CYT-PAP/Data Files', 'RA-CYT-PAP/Google Translate - !GYN Convential total list_유현주 (1) (1).xlsx']


In [7]:
from pathlib import Path

[f for f in list(fs.glob('s3://RA-CYT-PAP/**')) if Path(f).suffix in ['.svs','.ndpi']]

['RA-CYT-PAP/Data Files/GYN/Conventional/QC_GYN_16/C-16/C-16-011 - 2021-07-19 160706.ndpi',
 'RA-CYT-PAP/Data Files/GYN/Conventional/QC_GYN_16/C-16/C-16-012.svs',
 'RA-CYT-PAP/Data Files/GYN/Conventional/QC_GYN_16/C-16/C-16-013.svs',
 'RA-CYT-PAP/Data Files/GYN/Conventional/QC_GYN_16/C-16/C-16-014.svs',
 'RA-CYT-PAP/Data Files/GYN/Conventional/QC_GYN_16/C-16/C-16-017.svs',
 'RA-CYT-PAP/Data Files/GYN/Conventional/QC_GYN_16/C-16/C-16-021.svs',
 'RA-CYT-PAP/Data Files/GYN/Conventional/QC_GYN_16/C-16/C-16-025.svs',
 'RA-CYT-PAP/Data Files/GYN/Conventional/QC_GYN_16/C-16/C-16-026 - 2021-07-07 202640.ndpi',
 'RA-CYT-PAP/Data Files/GYN/Conventional/QC_GYN_16/C-16/C-16-038 - 2021-07-07 213311.ndpi',
 'RA-CYT-PAP/Data Files/GYN/Conventional/QC_GYN_16/C-16/C-16-044 - 2021-07-08 032245.ndpi',
 'RA-CYT-PAP/Data Files/GYN/Conventional/QC_GYN_16/C-16/C-16-046 - 2021-07-07 210338.ndpi',
 'RA-CYT-PAP/Data Files/GYN/Conventional/QC_GYN_16/C-16/C-16-048 - 2021-07-07 223936.ndpi',
 'RA-CYT-PAP/Data File

In [8]:
fs.get_file('s3://RA-CYT-PAP/Data Files/GYN/Conventional/QC_GYN_16/C-16/C-16-098.svs', '/root/feature-map-experiments/WSI',)

In [11]:
import boto3

# Создаем сессию S3
    # anon=False,
    # use_ssl=True,
    # endpoint_url='https://s3.us-west-002.backblazeb2.com',
    # key='002fcb156df54ab0000000003',
    # secret = 'K002eQNcQEL2F612vYfbBOFFFAwzf/g',
s3 = boto3.Session(

    aws_access_key_id='002fcb156df54ab0000000003',
    aws_secret_access_key='K002eQNcQEL2F612vYfbBOFFFAwzf/g',
)

# Создаем клиент S3
s3_client = s3.client('s3',
    endpoint_url='https://s3.us-west-002.backblazeb2.com',
)

In [15]:
import math
import os
import time
from boto3.s3.transfer import TransferConfig

# Code from: https://stackoverflow.com/a/56692099/3533930

def s3_get_meta_data(conn, bucket, key):
    meta_data = conn.head_object(
        Bucket=bucket,
        Key=key
    )
    return meta_data


def s3_download(conn, bucket, key, file_to, parallel_threads):
    start = time.time()
    md = s3_get_meta_data(conn, bucket, key)
    chunk = get_cunks(md["ContentLength"], parallel_threads)
    print("Making %s parallel s3 calls with a chunk size of %s each..." % (
        parallel_threads, convert_size(chunk))
    )
    # cur_dir = os.path.dirname(os.path.realpath(__file__))
    conn.download_file(
        Bucket=bucket,
        Key=key,
        Filename=file_to,
        Config=TransferConfig(
            max_concurrency=parallel_threads
        )
    )
    end = time.time() - start
    print("Finished downloading %s in %s seconds" % (key, end))


def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return "%s %s" % (s, size_name[i])


def get_cunks(size_bytes, desired_sections):
    return size_bytes / desired_sections

In [19]:
s3_download(s3_client, "RA-CYT-PAP", "Data Files/GYN/Conventional/QC_GYN_16/C-16/C-16-098.svs", str(workspace_dir / 'WSI' / 'C-16-098.svs'), 40)

Making 40 parallel s3 calls with a chunk size of 88.55 MB each...
Finished downloading Data Files/GYN/Conventional/QC_GYN_16/C-16/C-16-098.svs in 38.44276404380798 seconds
